In [110]:
import pandas as pd
import geopandas as gp
import os
import json
from datetime import datetime

In [113]:
df10 = pd.read_csv("Ref10_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df11 = pd.read_csv("Ref11_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df12 = pd.read_csv("Ref12_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df13 = pd.read_csv("Ref13_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df14 = pd.read_csv("Ref14_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )

In [114]:
files = [df10, df11, df12, df13, df14]

for df in files:
    df['count'] = 1

####Group by SICD Codes

In [115]:
df10_grouped = df10.groupby(['PRMSIC', 'PRMSICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

####Append NAICS Labels (For Both SICD and PNATITL Groupings)

In [116]:
grouped_files = [df10_grouped, df11_grouped, df12_grouped, df13_grouped, df14_grouped]

In [117]:
cols = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

In [118]:
def getNAICS(row):
    code = str(row['PNACODE'])[0:2]
    return code

In [119]:
def getNAICSLabel(row):
    label = naicsCODES[row['NAICS']] 
    return label

In [120]:
naicsCODES = \
{'11': 'AGRICULTURE, FORESTRY, FISHING AND HUNTING',
 '21': 'MINING',
 '22': 'UTILITIES',
 '23': 'CONSTRUCTION',
 '31': 'MANUFACTURING',
 '32': 'MANUFACTURING',
 '33': 'MANUFACTURING',
 '42': 'WHOLESALE TRADE',
 '44': 'RETAIL TRADE',
 '45': 'RETAIL TRADE',
 '48': 'TRANSPORTATION AND WAREHOUSING',
 '49': 'TRANSPORTATION AND WAREHOUSING',
 '51': 'INFORMATION',
 '52': 'FINANCE AND INSURANCE',
 '53': 'REAL ESTATE RENTAL AND LEASING',
 '54': 'PROFESSIONAL, SCIENTIFIC, AND TECHNICAL SERVICES',
 '55': 'MANAGEMENT OF COMPANIES AND ENTERPRISES',
 '56': 'ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIATION SERVICES',
 '61': 'EDUCATIONAL SERVICES',
 '62': 'HEALTH CARE AND SOCIAL ASSISTANCE',
 '71': 'ARTS, ENTERTAINMENT, AND RECREATION',
 '72': 'ACCOMMODATION AND FOOD SERVICES',
 '81': 'OTHER SERVICES (EXCEPT PUBLIC ADMINISTRATION)',
 '92': 'PUBLIC ADMINISTRATION',
 '99': 'UNCLASSIFIED'}

In [121]:
for df in grouped_files:
    df.columns = df.columns.droplevel(1)
    df.columns = cols 
    del df['count_MED']
    df.loc[:,'NAICS'] = df.apply(getNAICS, axis = 1)
    df.loc[:,'NAICS_LABEL'] = df.apply(getNAICSLabel, axis = 1)

####Upload Borough Shapefile Data

In [122]:
boros = gp.GeoDataFrame.from_file("../nyct2010_15b/nyct2010.shp")

####Append Shapefile Data

In [123]:
import shapefile
from pyproj import Proj, transform 
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon

In [124]:
def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x2,y2 = transform(inProj,outProj,x0,y0)
    
    return (x2, y2)

In [125]:
def convertCoordinates(row):
    geo = row['geometry']
    
    try: 
        x = list(geo.exterior.coords)
        new_list = [] 
    
        for i in x:
            new_list.append(convert(i))
        
        return Polygon(new_list)
    
    except: 
        x0 = list(geo[0].exterior.coords)
        x1 = list(geo[1].exterior.coords) 

        new_list0 = []
        new_list1 = []

        for i in x0:
            new_list0.append(convert(i))
    
        for j in x1:
            new_list1.append(convert(j))
    
        return MultiPolygon((Polygon(new_list0), Polygon(new_list1)))

####Create Directories with Files

In [128]:
YEARS = ["2010", "2011", "2012", "2013", "2014"]
NAICS = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(0))
PNATITL = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(1))
SICD = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(2))

In [129]:
cnt = len(SICD)
base_loc = '/gpfs1/cusp/kl1771/nycem/Ref_LMI/geojson'

In [130]:
for year in YEARS:
    year_folder = str(year)
    
    if year_folder in os.listdir(base_loc):
        print 'already created'
    else:
        add_year = base_loc + "/" + year_folder
        os.mkdir(add_year)

already created
already created
already created
already created
already created


In [146]:
df = df14_grouped #grouped_files[0]
year = "2014" # YEARS[0] 

base_year_loc = base_loc + "/" + year
year_dir = os.listdir(base_year_loc)

In [147]:
def createGeoJSON(pnatitl_loc, string2):
    
    #pnatitl_loc = base_year_loc + "/" + string1
    sicd_code = string2.replace("/", "-")
    sicd_file_name = sicd_code.replace("_", "").replace(", ", "_").replace(" ", "_").replace("(", "").\
    replace(")", "").lower()
    

    df_temp = df[df['SICD'] == string2]
    
    result = pd.merge(df_temp, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
        
    result = result[['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS_LABEL', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
                 'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'NTAName', 'Shape_Area', 'geometry']]
    
    result.columns = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS', 'Borough', 'Census_Tract', \
                  'Median_Income_ct', 'EMPSDT_Sum', 'EMPSDT_Median', 'SLSVDT_Sum', 'SLSVDT_Median', \
                  'Business_Count', 'Neighborhood_Name', 'Shape_Area', 'geometry']
    
    result.loc[:,'geometry'] = result.apply(convertCoordinates, axis = 1)
    geojson_string = gp.GeoDataFrame(result).to_json()
    geojson_dict = json.loads(geojson_string)

    loc = pnatitl_loc + "/" + sicd_file_name + ".geojson"
    
    with open(loc, 'wb') as fp:
        json.dump(geojson_dict, fp)

In [82]:
healthSPA = df10_grouped[df10_grouped['SICD'] == 'HEALTH SPAS']

result = pd.merge(healthSPA, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                   left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])

In [83]:
result = result[['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS_LABEL', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
                 'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'NTAName', 'Shape_Area', 'geometry']]

In [85]:
result.columns = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS', 'Borough', 'Census_Tract', \
                  'Median_Income_ct', 'EMPSDT_Sum', 'EMPSDT_Median', 'SLSVDT_Sum', 'SLSVDT_Median', \
                  'Business_Count', 'Neighborhood_Name', 'Shape_Area', 'geometry']

In [148]:
startTime = datetime.now()

for index in range(cnt): 
    
    year_dir = os.listdir(base_year_loc)
    
    naics = NAICS[index]
    naics_dir_name = naics.replace("_", "").replace(", ", "_").replace(" ", "_").replace("(", "").\
    replace(")", "").lower()
    
    pnatitl = PNATITL[index]
    pnatitl_dir_name = pnatitl.replace("/", "_").replace(" ", "_").replace("&", "and")\
    .replace("(", "").replace(")", "").replace(",", "_").replace("-", "_").replace("'","").lower()
    
    sicd = SICD[index]
    
    if naics_dir_name not in os.listdir(base_year_loc): 
        naics_loc = base_year_loc + "/" + naics_dir_name
        os.mkdir(naics_loc)
        
        pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
        os.mkdir(pnatitl_loc)
        createGeoJSON(pnatitl_loc, sicd) 

        
    else: 
        naics_loc = base_year_loc + "/" + naics_dir_name
        
        if pnatitl_dir_name not in os.listdir(naics_loc):
            pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
            os.mkdir(pnatitl_loc)
            createGeoJSON(pnatitl_loc, sicd) 
          
        else: 
            pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
            createGeoJSON(pnatitl_loc, sicd) 
            
print datetime.now() - startTime

0:22:29.156868


In [158]:
SICD[-10:]

['HANDBAGS-WHOLESALE',
 'HOSIERY-WHOLESALE',
 'LINGERIE-WHOLESALE',
 'MATERNITY APPAREL-WHOLESALE',
 'MILLINERY-WHOLESALE',
 'SWIMWEAR & ACCESSORIES-WHOLESALE',
 "UNIFORMS-MEN'S & WOMEN'S-WHOLESALE",
 "WOMEN'S ACCESSORIES-WHOLESALE",
 "WOMEN'S APPAREL-WHOLESALE",
 "WOMEN'S COATS & SUITS-WHOLESALE"]

In [163]:
PNATITL[-100:]
#NAICS[-100:]

['STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'STATIONERY & OFFICE SUPPLIES MERCHANT WHOLESALERS',
 'TIRE & TUBE MERCHANT WHOLESALERS',
 'TIRE & TUBE MERCHANT WHOLESALERS',
 'TIRE & TUBE MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUPPLIES MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUPPLIES MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUPPLIES MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUPPLIES MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUPPLIES MERCHANT WHOLESALERS',
 'TOY & HOBBY GOODS & SUP